# Analyzing Messages from NextDoor.com



First let's read in the messages...

In [5]:
import os, sys
import re

# Open a file
path = "./data/plainmessages"

len(dirs)

11125

In [91]:
# this regular expression fill match with strings that have only digits followed by a dot and 'msg'
only_msg = re.compile('(\d+)\.msg')

In [82]:
## function to extract data from message file
def parsemessage(filename):
    ph0 = re.compile("^\(Message")
    ph1 = re.compile("^([\w\-]+):\s+(.*)")
    ph2 = re.compile("^\s(.*)")
    ph3 = re.compile("^part\s+(\d+)\s+text/plain")
    ##ph4 = re.compile("^(\w+) (\w+) from ([\w\s]+)said:")
    ph4 = re.compile("^(\w+) (\w+) from (.+)")
    ph5 = re.compile("^Shared to (\d+) neighborhood")
    ##Shared to your neighborhood
    ph5a = re.compile("^Shared to your neighborhood")
    ph6 = re.compile("^To view")
    
    resdict = {}
    content = ""
    currentkey = ""
    state = "begin"
    blanklines = 0
    error_message = ""
    for line in open(filename):
        
        ##print("'%s' (%s)" % (line.rstrip(), state))
        res0 = ph0.match(line)
        res1 = ph1.match(line)
        res2 = ph2.match(line)
        res3 = ph3.match(line)
        res4 = ph4.match(line)
        res5 = ph5.match(line)
        res5a = ph5a.match(line)
        res6 = ph6.match(line)
        
        if state == "begin":
            if res0:
                state = "inheader"
                continue
            else:
                error_message = "problems in state %s" % state
                state = "error"
       
        if state == "inheader":
            if res1:
                currentkey = res1.group(1)
                resdict[currentkey] = res1.group(2)
                continue
            elif line[0]=='\n':
                continue
            elif line[0]=='\t':
                resdict[currentkey] += line
                continue
            elif res3:
                state = "part"
                continue
            else:
                error_message = "problems in state %s" % state
                state = "error"

        elif state=="part":
            if line[0]=="\n":
                continue
            elif res4:
                resdict["sender_firstname"] = res4.group(1)
                resdict["sender_lastname"] = res4.group(2)
                resdict["sender_neighborhood"] = res4.group(3)
                state = "body"
                continue
            else:
                error_message = "problems in state %s" % state
                state = "error"

        elif state=="body":
            if res5:
                resdict["Num_neighborhoods"] = res5.group(1)
                state = "last"
                continue
            elif res5a:
                resdict["Num_neighborhoods"] = 1
                state = "last"
                continue
            else:
                if "body" in resdict.keys():
                    resdict["body"] += line
                else:
                    resdict["body"] = line
                continue
        elif state=="last":
            if res6:
                state = "done"
                continue
            else:
                if "category" in resdict.keys():
                    resdict["category"] += line.rstrip()
                else:
                    resdict["category"] = line.rstrip()
                continue
        
        elif state=="done":
            break;
            
        else:
            ##print(error_message)
            return resdict
                
    return resdict

In [71]:
res = parsemessage("data/plainmessages/1000.msg")

In [66]:
res.keys()

dict_keys(['Subject', 'DKIM-Signature', 'Date', 'category', 'Reply-to', 'Authentication-Results', 'Received-SPF', 'Delivered-To', 'Num_neighborhoods', 'From', 'sender_lastname', 'sender_neighborhood', 'Status', 'X-SG-EID', 'To', 'body', 'X-Feedback-ID', 'X-Received', 'sender_firstname', 'X-SG-ID', 'MIME-Version', 'Return-Path'])

In [25]:
res['body']

'\nAny interest in modern glass-top C&B desk?\n\nOriginal post by Susan Bird from Oakhurst (3 replies):\n\n> Looking for crate and barrel/ modern style\n> Dining room table \n> Also looking for bookcase dressers \n> Rugs couches queen size mattress box spring\n> Washer/dryer out door furniture\n> Please just email pics...\n\n'

In [53]:
%more data/plainmessages/1000.msg


In [2]:
dirs[0:10]

['.DS_Store',
 '.mh_sequences.msg',
 '1.msg',
 '10.msg',
 '100.msg',
 '1000.msg',
 '10000.msg',
 '10001.msg',
 '10002.msg',
 '10003.msg']

In [99]:
# This would print all the files and directories
dirs = os.listdir( path )

msg_data = []
msg_target = []

for file in dirs:
    if only_msg.match(file):
        filepath = "%s/%s" % (path, file)
        ##print(filepath, end="\t")
        res = parsemessage(filepath)
        if 'body' in res.keys():
            msg_data.append(res['body'])
            if ('category' in res.keys()) and (len(res['category']) > 0):
                msg_target.append(res['category'])
            else:
                msg_target.append('unknown')
        # print("%s -> %s" % (file, res['category'])

In [100]:
len(msg_data), len(msg_target)

(9677, 9677)

In [103]:
len(set(msg_target))

69

In [111]:
import itertools
[(g[0], len(list(g[1]))) for g in itertools.groupby(msg_target)]

[('Lost & Found', 1),
 ('unknown', 67),
 ('General', 1),
 ('unknown', 23),
 ('Carpet cleaner', 1),
 ('unknown', 13),
 ('General', 1),
 ('unknown', 86),
 ('Crime & Safety', 1),
 ('unknown', 10),
 ('General', 1),
 ('unknown', 55),
 ('Classifieds', 1),
 ('unknown', 3),
 ('Classifieds', 1),
 ('unknown', 7),
 ('Classifieds', 1),
 ('unknown', 57),
 ('General', 1),
 ('unknown', 62),
 ('Free items', 1),
 ('unknown', 77),
 ('Free items', 1),
 ('unknown', 9),
 ('Classifieds', 1),
 ('unknown', 8),
 ('General', 1),
 ('unknown', 23),
 ('Recommendations', 1),
 ('unknown', 60),
 ('Crime & Safety', 1),
 ('unknown', 70),
 ('General', 1),
 ('unknown', 75),
 ('Handyman', 1),
 ('unknown', 4),
 ('General', 1),
 ('unknown', 2),
 ('Plumber', 1),
 ('unknown', 7),
 ('Classifieds', 1),
 ('unknown', 1),
 ('Classifieds', 1),
 ('Recommendations', 1),
 ('unknown', 13),
 ('General', 1),
 ('unknown', 1),
 ('General', 1),
 ('unknown', 8),
 ('General', 1),
 ('unknown', 12),
 ('Classifieds', 1),
 ('unknown', 2),
 ('Crim